# Phyling API

To use this notebook, you need to create a `.env` file with:

```env
PHYLING_API_KEY=your_api_key

# Server API:   https://api.app.phyling.fr
# Maxi-Hub API: 192.168.1.42:5001
PHYLING_API_URL="https://api.app.phyling.fr"
```

In [1]:
import json
import time
import logging
import os
from dotenv import load_dotenv

from phyling.api import PhylingAPI
from phyling.api import PhylingRealtime

load_dotenv()


True

# API connection

In [2]:
# Create API and Realtime instances
api = PhylingAPI(
    api_key=os.getenv("PHYLING_API_KEY"),
    url=os.getenv("PHYLING_API_URL", "https://api.app.phyling.fr/")
)

realtime = PhylingRealtime(api)

In [3]:
# Start auto-updating device list every socketIO update
realtime.autoUpdateDeviceList(True)

In [4]:
# Print the device list (will update automatically in the background)
print(realtime.getDeviceList())

[{'number': 10300378, 'name': 'Maxi 378', 'is_connected': True, 'state': 'record'}]


In [5]:
# Enable auto-updates for device status, indicator, and data for the first connected device
number = realtime.getDeviceList()[0]["number"]
realtime.autoUpdateDeviceStatus(number=number, enabled=True)
realtime.autoUpdateDeviceIndicator(number=number, enabled=True)
realtime.autoUpdateDeviceData(number=number, enabled=True)

In [6]:
# Print the device status (will update automatically in the background)
print(realtime.getDeviceStatus(number=number))

{'battery': 41, 'client': {'id': 2, 'name': 'phyling'}, 'deviceRecId': 214, 'epoch': 1763629756.0, 'features': {'calib': {'cmd_get': 'v1.board.calib.get', 'cmd_set': 'v1.board.calib.set'}, 'config': {'cmd_get': 'v1.board.config.get', 'cmd_set': 'v1.board.config.set'}, 'device_info': {'cmd_get': 'v1.board.device_info.get', 'cmd_set': 'v1.board.device_info.set'}, 'firmware_update': {'cmd': 'v1.board.firmware_update'}, 'logs': {'cmd_get': 'v1.logs.get', 'cmd_list': 'v1.logs.list'}, 'record': {'cmd_start': 'v1.record.rec.start', 'cmd_stop': 'v1.record.rec.stop'}, 'restart': {'cmd': 'v1.board.restart'}, 'scenario': {'cmd_start': 'v1.record.scenario.start', 'cmd_stop': 'v1.record.scenario.stop'}, 'shutdown': {'cmd': 'v1.board.shutdown'}, 'time_set': {'cmd': 'v1.board.time.set'}, 'wifi': {'cmd_get': 'v1.board.wifi.get', 'cmd_set': 'v1.board.wifi.set'}}, 'group': {'id': 1, 'name': 'phyling'}, 'is_connected': True, 'modules': {'gps': {'id': 1, 'is_connected': True, 'realtime': True, 'type': 'gp

In [7]:
# Print the device state only
print(realtime.getDeviceStatus(number=number)["state"])

record


In [8]:
# Print the available RPC methods
print(realtime.getDeviceStatus(number=number)["features"])

{'calib': {'cmd_get': 'v1.board.calib.get', 'cmd_set': 'v1.board.calib.set'}, 'config': {'cmd_get': 'v1.board.config.get', 'cmd_set': 'v1.board.config.set'}, 'device_info': {'cmd_get': 'v1.board.device_info.get', 'cmd_set': 'v1.board.device_info.set'}, 'firmware_update': {'cmd': 'v1.board.firmware_update'}, 'logs': {'cmd_get': 'v1.logs.get', 'cmd_list': 'v1.logs.list'}, 'record': {'cmd_start': 'v1.record.rec.start', 'cmd_stop': 'v1.record.rec.stop'}, 'restart': {'cmd': 'v1.board.restart'}, 'scenario': {'cmd_start': 'v1.record.scenario.start', 'cmd_stop': 'v1.record.scenario.stop'}, 'shutdown': {'cmd': 'v1.board.shutdown'}, 'time_set': {'cmd': 'v1.board.time.set'}, 'wifi': {'cmd_get': 'v1.board.wifi.get', 'cmd_set': 'v1.board.wifi.set'}}


In [9]:
# Perform a RPC request
realtime.executeRPC(
    number=number,
    method="v1.record.rec.start",
    params={}
)

In [10]:
# Print the device indicator (will update automatically in the background - only on record/scenario states)
print(realtime.getDeviceIndicator(number=number))

{'number': 10300378, 'recTime': 51.959, 'indicators': {'cadence': {'x': 51.678889, 'y': 0}, 'T_500m': {'x': 51.559823, 'y': 'NaN'}}}


In [11]:
# Print the latest realtime data (will update automatically in the background - only on record/scenario states)
lastRtData = realtime.getDeviceData(number=number)
print(lastRtData)

{'number': 10300378, 'recTime': 56.939, 'data': {'imu': {'acc_x': [0.359046, 0.363833, 0.301599, 0.330322, 0.31596, 0.320748], 'acc_y': [0.215428, 0.191491, 0.205853, 0.205853, 0.215428, 0.186704], 'acc_z': [9.952756, 9.914457, 9.866585, 9.967117, 9.924032, 9.919245], 'gyro_x': [2.59, 2.66, 2.66, 2.66, 2.59, 2.73], 'gyro_y': [-4.97, -4.97, -4.97, -4.83, -4.83, -4.9], 'gyro_z': [-1.96, -1.96, -2.03, -1.96, -1.96, -1.96], 'T': [56.673, 56.693, 56.713, 56.733, 56.753, 56.773]}, 'gps': {'gpstimeUs': [1763629770799947], 'gpsTimeAccuracyNs': [268], 'longitude': [2.098548], 'latitude': [48.731577], 'speed': [0.018], 'altitude': [138.574997], 'heading': [271.869629], 'nSat': [4], 'PDOP': [11.81], 'T': [56.744]}}, 'selections': []}


In [13]:
# Continuously print new realtime data as it arrives
ONLY_FIRST = True
lastRecTime = 0
while True:
    status = realtime.getDeviceStatus(number=number)
    rtData = realtime.getDeviceData(number=number)
    now = time.time()
    if rtData["recTime"] <= lastRecTime:
        continue
    lastRecTime = rtData["recTime"]
    dataStr = f"[{rtData['recTime']:6.3f}] "
    for modname, modcontent in rtData.get("data", {}).items():
        latency = now - status['startRecTimeSinceEpoch'] - rtData["data"][modname]["T"][-1]
        dataStr +=  f" | latency: {int(latency * 1000):4d}ms"
        for key, value in modcontent.items():
            if key in ["T"]:
                continue
            if len(dataStr) > 0:
                dataStr += " | "
            dataStr += f"{modname}.{key}: {value[-1]:.2f}"
            if ONLY_FIRST:
                break
    if len(dataStr) > 0:
        print(dataStr)
    time.sleep(0.01)

[114.827]  | latency:  236ms | imu.acc_x: 0.28
[114.994]  | latency:  259ms | imu.acc_x: 0.28 | latency:  333ms | gps.gpstimeUs: 1763629828799895.00
[115.113]  | latency:  214ms | imu.acc_x: 0.30 | latency:  212ms | gps.gpstimeUs: 1763629828999896.00
[115.229]  | latency:  208ms | imu.acc_x: 0.30
[115.402]  | latency:  262ms | imu.acc_x: 0.24 | latency:  336ms | gps.gpstimeUs: 1763629829199895.00
[115.519]  | latency:  219ms | imu.acc_x: 0.33 | latency:  328ms | gps.gpstimeUs: 1763629829299895.00
[115.642]  | latency:  215ms | imu.acc_x: 0.28 | latency:  249ms | gps.gpstimeUs: 1763629829499895.00
[115.763]  | latency:  214ms | imu.acc_x: 0.31 | latency:  283ms | gps.gpstimeUs: 1763629829599895.00
[115.927]  | latency:  276ms | imu.acc_x: 0.30 | latency:  360ms | gps.gpstimeUs: 1763629829699895.00
[116.051]  | latency:  248ms | imu.acc_x: 0.48 | latency:  282ms | gps.gpstimeUs: 1763629829899895.00
[116.171]  | latency:  248ms | imu.acc_x: 0.32 | latency:  272ms | gps.gpstimeUs: 17636298

KeyboardInterrupt: 

In [ ]:
# Continuously print new realtime indicators as they arrive
# {'number': 10200300, 'recTime': 178.123, 'indicators': {'acc_x': {'x': 177.842491, 'y': 0.2968113739043474}}}
ONLY_FIRST = False
lastRecTime = 0
while True:
    status = realtime.getDeviceStatus(number=number)
    rtIndicators = realtime.getDeviceIndicator(number=number)
    now = time.time()
    if rtIndicators["recTime"] <= lastRecTime:
        continue
    lastRecTime = rtIndicators["recTime"]
    dataStr = f"[{rtIndicators['recTime']:6.3f}] "
    for indname, indcontent in rtIndicators.get("indicators", {}).items():
        latency = now - status['startRecTimeSinceEpoch'] - rtIndicators["indicators"][indname]["x"]
        dataStr +=  f"latency: {int(latency * 1000):4d}ms {indname}: {indcontent['y']:.2f}"
        if ONLY_FIRST:
            break
    if len(dataStr) > 0:
        print(dataStr)
    time.sleep(0.01)

[382.254] latency:  899ms acc_x: 0.29
[382.840] latency:  370ms acc_x: 0.31
[383.373] latency:  337ms acc_x: 0.31
[384.025] latency:  338ms acc_x: 0.30
[384.564] latency:  337ms acc_x: 0.32
[385.082] latency:  347ms acc_x: 0.32
[385.730] latency:  339ms acc_x: 0.51
[386.391] latency:  364ms acc_x: -9.34
[386.999] latency:  298ms acc_x: -9.11
[387.524] latency:  301ms acc_x: -8.89
[388.079] latency:  315ms acc_x: -9.15
[388.593] latency:  332ms acc_x: -1.32
[389.138] latency:  329ms acc_x: -1.35
[389.796] latency:  339ms acc_x: -1.33
[390.353] latency:  368ms acc_x: -1.26
[390.860] latency:  349ms acc_x: -1.20
[391.473] latency:  320ms acc_x: -1.02
[392.022] latency:  310ms acc_x: -0.82
[392.555] latency:  323ms acc_x: -0.63
[393.060] latency:  306ms acc_x: -3.08
[393.606] latency:  325ms acc_x: -8.37
[394.395] latency:  847ms acc_x: -8.73
[394.901] latency:  336ms acc_x: -7.11
[395.409] latency:  322ms acc_x: 9.62
[395.949] latency:  329ms acc_x: 0.26
[396.613] latency:  331ms acc_x: 0

KeyboardInterrupt: 

In [ ]:
realtime.executeRPC(
    number=number,
    method="v1.record.rec.stop",
    params={},
    wait_for_result=False
)